#### Setup for Google Colab (Optional)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

##### UPDATE IT IF NEEDED

In [ ]:
cd 'drive/MyDrive/Colab Notebooks/comparative-study-bgru-gan-model'

In [ ]:
ls

# Hyper parameter searching with keras-tuner
Train BGRU models for HK and US stock market, repectively
<p>
ref: https://www.tensorflow.org/tutorials/keras/keras_tuner

```bash
conda install keras-tuner
```

In [12]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, Bidirectional
from tensorflow.keras.layers import GRU
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.metrics import Accuracy, MeanAbsoluteError, RootMeanSquaredError, MeanAbsolutePercentageError, MeanSquaredError
from keras.callbacks import ModelCheckpoint, EarlyStopping

import keras_tuner as kt

### Common variables and functions

In [13]:
MODEL_STRUCTURE_PATH = "./diagrams/model/structures"
MODEL_TRAIN_HISTORY_DIAGRAMS_PATH = "./diagrams/model/training"
PROCESSED_STOCKS_PATH = "./data/processed/training_data"
TRAINING_STOCKS_PATH = "./data/processed/training_data"
EVALUATE_STOCKS_PATH = "./data/processed/stocks_for_evaluate"
TRAIN_STOCK_NAMES_PATH = "./data/processed/stock_names_for_training"

# stocks model checkpoint paths
HK_MODELS_CHECKPOINT_PATH = "./model/hk"
US_MODELS_CHECKPOINT_PATH = "./model/us"

hk_bgru_file_path = "{}/bgru.h5".format(HK_MODELS_CHECKPOINT_PATH)
hk_bgru_train_history_file_path = "{}/bgru_training_history.npy".format(HK_MODELS_CHECKPOINT_PATH)
# hk_bgru_train_history_file_path = "{}/bgru_history/{}_bgru_training_history.npy"

us_bgru_file_path = "{}/bgru.h5".format(US_MODELS_CHECKPOINT_PATH)
us_bgru_train_history_file_path = "{}/bgru_training_history.npy".format(US_MODELS_CHECKPOINT_PATH)

TRAIN_EPOCHS = 100

def create_dir_if_not_exist(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname, exist_ok=True)

### Import datasets

In [14]:
# stock names
# template_filename_train_x = "{}/{}_train_X.npy"
# template_filename_train_y = "{}/{}_train_y.npy"
#
# template_filename_test_x = "{}/{}_test_X.npy"
# template_filename_test_y = "{}/{}_test_y.npy"
#
# fns_hk = np.load("{}/hk_train_stock_names.npy".format(TRAIN_STOCK_NAMES_PATH))
# fns_us = np.load("{}/us_train_stock_names.npy".format(TRAIN_STOCK_NAMES_PATH))
#
# X_train_hk = {}
# y_train_hk = {}
# X_test_hk = {}
# y_test_hk = {}
# for i in range(len(fns_hk)):
#     X_train_hk[fns_hk[i]] = np.load(template_filename_train_x.format(
#         TRAINING_STOCKS_PATH,
#         fns_hk[i]
#     ))
#
#     y_train_hk[fns_hk[i]] = np.load(template_filename_train_y.format(
#         TRAINING_STOCKS_PATH,
#         fns_hk[i]
#     ))
#
#     X_test_hk[fns_hk[i]] = np.load(template_filename_test_x.format(
#         TRAINING_STOCKS_PATH,
#         fns_hk[i]
#     ))
#
#     y_test_hk[fns_hk[i]] = np.load(template_filename_test_y.format(
#         TRAINING_STOCKS_PATH,
#         fns_hk[i]
#     ))
#
# X_train_us = {}
# y_train_us = {}
# X_test_us = {}
# y_test_us = {}
# for i in range(len(fns_us)):
#     X_train_us[fns_us[i]] = np.load(template_filename_train_x.format(
#         TRAINING_STOCKS_PATH,
#         fns_us[i]
#     ))
#
#     y_train_us[fns_us[i]] = np.load(template_filename_train_y.format(
#         TRAINING_STOCKS_PATH,
#         fns_us[i]
#     ))
#
#     X_test_us[fns_us[i]] = np.load(template_filename_test_x.format(
#         TRAINING_STOCKS_PATH,
#         fns_us[i]
#     ))
#
#     y_test_us[fns_us[i]] = np.load(template_filename_test_y.format(
#         TRAINING_STOCKS_PATH,
#         fns_us[i]
#     ))
#
# # Check the imports, minus the one stock that used to test generalizability
# assert len(X_train_hk) == 49
# assert len(y_train_hk) == 49
# assert len(X_test_hk) == 49
# assert len(X_test_hk) == 49
#
# assert len(X_train_us) == 49
# assert len(y_train_us) == 49
# assert len(X_test_us) == 49
# assert len(X_test_us) == 49

# hk datasets
X_train_hk = np.load("{}/train_X_hk.npy".format(PROCESSED_STOCKS_PATH))
X_test_hk = np.load("{}/test_X_hk.npy".format(PROCESSED_STOCKS_PATH))
y_train_hk = np.load("{}/train_y_hk.npy".format(PROCESSED_STOCKS_PATH))
y_test_hk = np.load("{}/test_y_hk.npy".format(PROCESSED_STOCKS_PATH))

# us datasets
X_train_us = np.load("{}/train_X_us.npy".format(PROCESSED_STOCKS_PATH))
X_test_us = np.load("{}/test_X_us.npy".format(PROCESSED_STOCKS_PATH))
y_train_us = np.load("{}/train_y_us.npy".format(PROCESSED_STOCKS_PATH))
y_test_us = np.load("{}/test_y_us.npy".format(PROCESSED_STOCKS_PATH))

### Define models structure
##### BGRU models
###### Reference:
```
Salimath, S., Chatterjee, T., Mathai, T., Kamble, P., & Kolhekar, M. (2021, April). Prediction of Stock Price for Indian Stock Market: A Comparative Study Using LSTM and GRU. In International Conference on Advances in Computing and Data Sciences (pp. 292-302). Springer, Cham.
Lin, H., Chen, C., Huang, G., & Jafari, A. (2021). Stock price prediction using Generative Adversarial Networks. Journal of Computer Science, (17(3), 188–196. doi:10.3844/jcssp.2021.188.196
https://github.com/grudloff/stock_market_GAN
Train with multiple stocks: https://www.kaggle.com/humamfauzi/multiple-stock-prediction-using-single-nn
Priya, R. S., & Sruthi, C. (2022). Stock Price Prediction Based on Deep Learning Using Long Short-Term Memory. In Futuristic Communication and Network Technologies (pp. 67-76). Springer, Singapore.
```

In [15]:
input_dim = X_train_hk.shape[1]
feature_cnt = X_train_hk.shape[2]
def make_bgru_model(hp) -> tf.keras.models.Model:
    model = Sequential()

    # input layer
    model.add(
        Input(
            shape=(input_dim, feature_cnt)
        )
    )

    bgru_layer1_units = hp.Int('bgru_layer1_units', min_value=32, max_value=512, step=32)
    bgru_dropout1_rate = hp.Float("bgru_dropout1_rate", min_value=0, max_value=0.99, step=0.05)
    # first gru + dropout layer
    model.add(
        Bidirectional(
            GRU(
                units=bgru_layer1_units,
                return_sequences=True,
                input_shape=(input_dim, feature_cnt),
                activation="tanh"
                )
        )
    )
    model.add(
        # Dropout(rate=0.3)
        Dropout(rate=bgru_dropout1_rate)
    )

    bgru_layer2_units = hp.Int('bgru_layer2_units', min_value=32, max_value=512, step=32)
    bgru_dropout2_rate = hp.Float("bgru_dropout2_rate", min_value=0, max_value=0.99, step=0.05)
    # second gru + dropout layer
    model.add(
        Bidirectional(
            GRU(
                # units=64,
                units=bgru_layer2_units,
                return_sequences=False, # important, convert array from 3d to 2d
                input_shape=(input_dim, feature_cnt),
                activation="tanh"
                )
        )
    )
    model.add(
        # Dropout(rate=0.5)
        Dropout(rate=bgru_dropout2_rate)
    )

    # output dense layer
    model.add(
        Dense(units = 1)
    )

    hp_adam_lr = hp.Choice('hp_adam_lr', values=[1e-2, 1e-3, 1e-4, 1e-5])
    # compile model and use Adam optimizer
    model.compile(
        # optimizer=Adam(learning_rate=0.001),
        optimizer=Adam(learning_rate=hp_adam_lr),
        loss="mean_squared_error",
        metrics=[
            MeanAbsoluteError(),
            RootMeanSquaredError(),
            MeanAbsolutePercentageError()
        ]
    )

    print(model.summary())
    return model


### Start searching the model hyper-parameter, take HK model as example

In [17]:
tuner = kt.Hyperband(
    make_bgru_model,
    objective='val_loss',
    max_epochs=TRAIN_EPOCHS,
    factor=3,
    directory="model_hyperparam_search",
    project_name='bgru_hyperparam_search'
)

hk_bgru_es = EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=5,
    # min_delta=0.0001, # https://stackoverflow.com/a/63495687/9500852
)

tuner.search(
    X_train_hk,
    y_train_hk,
    validation_data=(X_test_hk, y_test_hk),
    callbacks=[hk_bgru_es]
)

best_hps = tuner.get_best_hyperparameters(num_trails=1)[0]

print(f"""
The hyperparameter search is complete. \n
bgru_layer1_units: {best_hps.get('bgru_layer1_units')} \n
bgru_dropout1_rate: {best_hps.get('bgru_dropout1_rate')} \n
bgru_layer2_units: {best_hps.get('bgru_layer2_units')} \n
bgru_dropout2_rate: {best_hps.get('bgru_dropout2_rate')} \n
hp_adam_lr: {best_hps.get('hp_adam_lr')} \n
""")

Trial 104 Complete [00h 05m 20s]
val_loss: 0.0024910131469368935

Best val_loss So Far: 0.001608289428986609
Total elapsed time: 12h 44m 05s

Search: Running Trial #105

Hyperparameter    |Value             |Best Value So Far 
bgru_layer1_units |224               |192               
bgru_dropout1_rate|0.2               |0.1               
bgru_layer2_units |448               |320               
bgru_dropout2_rate|0.45              |0.3               
hp_adam_lr        |0.001             |0.001             
tuner/epochs      |4                 |2                 
tuner/initial_e...|2                 |0                 
tuner/bracket     |4                 |4                 
tuner/round       |1                 |0                 
tuner/trial_id    |d0567797c78c283...|None              

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 30, 448)      

KeyboardInterrupt: 

In [20]:
best_hps = tuner.get_best_hyperparameters()[0]

print(f"""
The hyperparameter search is complete. \n
bgru_layer1_units: {best_hps.get('bgru_layer1_units')} \n
bgru_dropout1_rate: {best_hps.get('bgru_dropout1_rate')} \n
bgru_layer2_units: {best_hps.get('bgru_layer2_units')} \n
bgru_dropout2_rate: {best_hps.get('bgru_dropout2_rate')} \n
hp_adam_lr: {best_hps.get('hp_adam_lr')} \n
""")


The hyperparameter search is complete. 

bgru_layer1_units: 192 

bgru_dropout1_rate: 0.1 

bgru_layer2_units: 320 

bgru_dropout2_rate: 0.30000000000000004 

hp_adam_lr: 0.001 




In [ ]:
# hk_bgru_cp = ModelCheckpoint(
#     filepath=hk_bgru_file_path,
#     monitor="val_loss",
#     save_best_only=True,
#     verbose=1,
#     mode="min"
# )
#
#
#
# hk_bgru_model = None
# # check if we have previously trained model or not, ref: https://stackoverflow.com/a/56425146/9500852
# if os.path.exists(hk_bgru_file_path):
#     print("Found existing model")
#     hk_bgru_model = load_model(hk_bgru_file_path)
#     # score = hk_bgru_model.evaluate()
# else:
#     print("No existing model is found")
#     hk_bgru_model = make_bgru_model(
#         input_dim=X_train_hk.shape[1],
#         feature_cnt=X_train_hk.shape[2]
#     )

# start fitting the model
# hk_bgru_history = None
#
# for i in range(len(fns_hk)):
#     print("\n--- Training {}, {} stocks remains ---".format(fns_hk[i], len(fns_hk) - i))
#     tmp_hk_bgru_history = hk_bgru_model.fit(
#         x=X_train_hk[fns_hk[i]],
#         y=y_train_hk[fns_hk[i]],
#         validation_data=(X_test_hk[fns_hk[i]], y_test_hk[fns_hk[i]]),
#         epochs=TRAIN_EPOCHS,
#         callbacks=[
#             hk_bgru_cp,
#             hk_bgru_es
#         ]
#     )
#
#     # Append to previous histories
#     if hk_bgru_history is None:
#         hk_bgru_history = tmp_hk_bgru_history.history
#     else:
#         for dict_key in hk_bgru_history.keys():
#             hk_bgru_history.update({
#                 dict_key: hk_bgru_history[dict_key] + tmp_hk_bgru_history.history[dict_key]
#             })
#
#     # Save model per fit() iteration
#     hk_bgru_model.save(hk_bgru_file_path)
#     print("After trained {}, Model Saved".format(fns_hk[i]))
#
#     # Save history per fit() iteration
#     np.save(hk_bgru_train_history_file_path, hk_bgru_history)
#     print("After trained {}, History Saved, loss len: {}".format(fns_hk[i], len(hk_bgru_history["loss"])))

    # SAVE HISTORY INDIVIUALLY PER fit() interation
    # np.save(hk_bgru_train_history_file_path.format(HK_MODELS_CHECKPOINT_PATH, fns_hk[i]), hk_bgru_history.history)
    # print("After trained {}, History Saved".format(fns_hk[i])))

# hk_bgru_history = hk_bgru_model.fit(
#     x=X_train_hk,
#     y=y_train_hk,
#     validation_data=(X_test_hk, y_test_hk),
#     epochs=TRAIN_EPOCHS,
#     callbacks=[
#         hk_bgru_cp,
#         hk_bgru_es
#     ]
# )
#
# # save the model
# if hk_bgru_model is not None:
#     hk_bgru_model.save(hk_bgru_file_path)
#     print("Model Saved")
#
# # save the training history
# if hk_bgru_history is not None:
#     # np.save(hk_bgru_train_history_file_path, hk_bgru_history.history)
#     np.save(hk_bgru_train_history_file_path, hk_bgru_history.history)
#     print("History Saved")

##### Train BGRU model for United States stocks

In [ ]:
# us_bgru_cp = ModelCheckpoint(
#     filepath=us_bgru_file_path,
#     monitor="val_loss",
#     save_best_only=True,
#     verbose=1,
#     mode="min"
# )
#
# us_bgru_es = EarlyStopping(
#     monitor="val_loss",
#     mode="min",
#     patience=5,
#     restore_best_weights=True, # added to prevent overfitting in iterative fit()
#     min_delta=0.0001, # https://stackoverflow.com/a/63495687/9500852
# )
#
# us_bgru_model = None
#
# # check if we have previously trained model or not, ref: https://stackoverflow.com/a/56425146/9500852
# if os.path.exists(us_bgru_file_path):
#     print("Found existing model")
#     us_bgru_model = load_model(us_bgru_file_path)
#
# else:
#     print("No existing model is found")
#     us_bgru_model = make_bgru_model(
#         input_dim=X_train_us[fns_us[0]].shape[1],
#         feature_cnt=X_train_us[fns_us[0]].shape[2]
#     )
#
# # start fitting the model
# us_bgru_history = None
#
# for i in range(len(fns_us)):
#     print("\n--- Training {}, {} stocks remains ---".format(fns_us[i], len(fns_us) - i))
#     tmp_us_bgru_history = us_bgru_model.fit(
#         x=X_train_us[fns_us[i]],
#         y=y_train_us[fns_us[i]],
#         validation_data=(X_test_us[fns_us[i]], y_test_us[fns_us[i]]),
#         epochs=TRAIN_EPOCHS,
#         callbacks=[
#             us_bgru_cp,
#             us_bgru_es,
#         ]
#     )
#
#     # Append to previous histories
#     if us_bgru_history is None:
#         us_bgru_history = tmp_us_bgru_history.history
#     else:
#         for dict_key in us_bgru_history.keys():
#             # us_bgru_history[dict_key] = us_bgru_history[dict_key] + tmp_us_bgru_history.history[dict_key]
#             us_bgru_history.update({
#                 dict_key: us_bgru_history[dict_key] + tmp_us_bgru_history.history[dict_key]
#             })
#
#     # Save model per fit() iteration
#     us_bgru_model.save(us_bgru_file_path)
#     print("After trained {}, Model Saved".format(fns_us[i]))
#
#     np.save(us_bgru_train_history_file_path, us_bgru_history)
#     print("After trained {}, History Saved".format(fns_us[i]))
#
# # save the model
# if us_bgru_model is not None:
#     us_bgru_model.save(us_bgru_file_path)
#     print("Model Saved")
#
# # save the training history
# if us_bgru_history is not None:
#     np.save(us_bgru_train_history_file_path, us_bgru_history)
#     print("History Saved")


### plot training history

In [ ]:
# def plot_history(history_dict, title):
#     """
#     Plot the training history
#     :param history_dict: dict, the training history, should be a dict (from keras' history.history)
#     :param title: str, plot title, example: "HK BGRU Model - {}", the program will replace the {} with the relevant metric name
#     :return:
#     """
#     metrics = ["loss",
#            "mean_absolute_error",
#            "root_mean_squared_error",
#            # "mean_absolute_percentage_error" # disable plot of MAPE as the normalized data consists of 0 or nearly 0, the MAPE is unreasonably high, will recalculate in evaluation stage
#            # "val_loss",
#            # "val_mean_absolute_error",
#            # "val_root_mean_squared_error",
#            # "val_mean_absolute_percentage_error"
#            ]
#
#     for metric in metrics:
#         plt.figure(figsize=(14, 5), dpi=500, facecolor="white")
#         # metrics.replace("_", "").title()
#         plt.plot(history_dict[metric], label="Training")
#         plt.plot(history_dict["val_{}".format(metric)], label="Validation")
#         plt.xlabel("Epochs")
#         plt.ylabel(metric.replace("_", " ").title())
#         plt.title(title.format(metric.replace("_", " ").title()))
#         plt.legend()
#         create_dir_if_not_exist(MODEL_TRAIN_HISTORY_DIAGRAMS_PATH)
#         # plt.savefig('{}/{}.png'.format(MODEL_TRAIN_HISTORY_DIAGRAMS_PATH, plt.gca().get_title()))
#         plt.show()

##### Plot HK BGRU training history

In [ ]:
# hk_bgru_history_dict = np.load(hk_bgru_train_history_file_path, allow_pickle=True).item()
# plot_history(hk_bgru_history_dict, "BGRU Model for HK Stock Price Predictions - {}")

In [ ]:
# hk_bgru_history_dict

##### Plot US BGRU training history

In [ ]:
# us_bgru_history_dict = np.load(us_bgru_train_history_file_path, allow_pickle=True).item()
# plot_history(us_bgru_history_dict, "BGRU Model for US Stock Price Predictions - {}")


### Other testing codes (to be removed)


In [ ]:
# Test train for one stock only

# hk_bgru_cp = ModelCheckpoint(
#     filepath=hk_bgru_file_path,
#     monitor="val_loss",
#     save_best_only=True,
#     verbose=1,
#     mode="min"
# )
#
# hk_bgru_es = EarlyStopping(
#     monitor="val_loss",
#     mode="min",
#     patience=10
# )
#
# hk_bgru_model = None
# # check if we have previously trained model or not, ref: https://stackoverflow.com/a/56425146/9500852
# if os.path.exists(hk_bgru_file_path):
#     print("Found existing model")
#     hk_bgru_model = load_model(hk_bgru_file_path)
#     # score = hk_bgru_model.evaluate()
# else:
#     print("No existing model is found")
#     hk_bgru_model = make_bgru_model(
#         input_dim=X_train_hk[fns_hk[0]].shape[1],
#         feature_cnt=X_train_hk[fns_hk[0]].shape[2]
#     )
#
# # from tensorflow.keras.layers import LSTM
# # hk_bgru_model = Sequential()
# # hk_bgru_model.add(Bidirectional(LSTM(units= 128), input_shape=(X_train_hk[fns_hk[0]].shape[1], X_train_hk[fns_hk[0]].shape[2])))
# # hk_bgru_model.add(Dense(64))
# # hk_bgru_model.add(Dense(units=1))
# #
# # hk_bgru_model.compile(optimizer=Adam(lr = 0.001), loss='mean_squared_error',
# #                   metrics=[
# #                     # MeanAbsoluteError(),
# #                     # RootMeanSquaredError(),
# #                     # MeanAbsolutePercentageError()
# #                 ])
#
# # print(hk_bgru_model.summary())
#
# print("\n--- Training {}, {} stocks remains ---".format("1038.HK", 0))
# hk_bgru_history = hk_bgru_model.fit(
#     x=X_train_hk["1038.HK"],
#     y=y_train_hk["1038.HK"],
#     validation_data=(X_test_hk["1038.HK"], y_test_hk["1038.HK"]),
#     epochs=TRAIN_EPOCHS,
#     # epochs=15,
#     # batch_size=150,
#     callbacks=[
#         hk_bgru_cp,
#         hk_bgru_es
#     ],
#     shuffle=False
# )
#
# hk_bgru_model.save(hk_bgru_file_path)
# print("After trained {}, Model Saved".format("1038.HK"))
# np.save(hk_bgru_train_history_file_path, hk_bgru_history.history)
# print("History Saved")

In [ ]:
# X_train_hk[fns_hk[0]].shape

In [ ]:
# hk_gru_history_dict = np.load(hk_bgru_train_history_file_path, allow_pickle=True).item()
#
# dict_key = "loss"
# print(len(hk_gru_history_dict["loss"]))
#
# # hk_bgru_history_dict["loss"] = hk_gru_history_dict["loss"] + hk_gru_history_dict["loss"]
# hk_bgru_history_dict.update({
#     dict_key: hk_gru_history_dict["loss"] + hk_gru_history_dict["loss"]
# })
#
# # print("updated")
#
# print(len(hk_bgru_history_dict["loss"]))
#
# print(hk_bgru_history_dict)

